In [35]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format

# Data Preparation

In [36]:
# Загрузка данных из файла
data_path = 'Final_Data.csv'
panel_data = pd.read_csv(data_path)

panel_data.drop(columns=['2022 [YR2022]', '2023 [YR2023]'], inplace=True)

# Просмотр первых нескольких строк данных
panel_data.head()

# Сначала создадим список колонок, которые нужно удалить
columns_to_drop = [f'{year} [YR{year}]' for year in range(1990, 2000)]

# Теперь удалим эти колонки из DataFrame
panel_data.drop(columns=columns_to_drop, inplace=True)

# Преобразование данных в "длинный" формат
panel_long = pd.melt(panel_data, id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
                     var_name="Year", value_name="Value")

# Изменение формата года, удалив "[YR" и "]" и конвертация в integer, если возможно
panel_long['Year'] = panel_long['Year'].str.extract('(\d{4})').astype(int)

# Преобразование данных с Series Name в качестве колонок
panel_pivot = panel_long.pivot_table(index=["Country Name", "Country Code", "Year"],
                                     columns="Series Name", values="Value", aggfunc='first')

# Сброс мульти-индекса колонок для упрощения структуры
panel_pivot.columns.name = None
panel_pivot.reset_index(inplace=True)

# Просмотр результатов преобразования
panel_pivot

,Country Name,Country Code,Year,Control of Corruption: Estimate,Current account balance (% of GDP),"Current account balance (BoP, current US$)","Foreign direct investment, net (BoP, current US$)","Foreign direct investment, net inflows (% of GDP)","Foreign direct investment, net inflows (BoP, current US$)",GDP growth (annual %),...,Individuals using the Internet (% of population),"Inflation, GDP deflator (annual %)","Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)",Labor force with advanced education (% of total working-age population with advanced education),Mobile cellular subscriptions (per 100 people),"Official exchange rate (LCU per US$, period average)",Political Stability and Absence of Violence/Terrorism: Estimate,Rule of Law: Estimate,"Unemployment, total (% of total labor force) (modeled ILO estimate)","Unemployment, total (% of total labor force) (national estimate)"
0,Afghanistan,AFG,2000,-1.27172422409058,..,..,..,..,170000,..,...,..,..,46.718,..,0,47.3575747311828,-2.43896889686584,-1.78066074848175,7.942,..
1,Afghanistan,AFG,2001,..,..,..,..,..,680000,..,...,0.004722568,..,46.754,..,0,47.500014516129,..,..,7.942,..
2,Afghanistan,AFG,2002,-1.25113701820374,..,..,..,1.30694988186642,50000000,..,...,0.004561395,..,46.812,..,0.119046168,47.263,-2.03503370285034,-1.67347323894501,7.935,..
3,Afghanistan,AFG,2003,-1.34417974948883,..,..,..,1.27849324104935,57799999.9,8.8322778126357,...,0.087891253,11.655238211175,46.881,..,0.883192104,48.7627535833333,-2.19837236404419,-1.5582937002182,7.929,..
4,Afghanistan,AFG,2004,-1.35064733028412,..,..,..,3.577104200584,186900000,1.41411798058977,...,0.10580903,11.271432071264,46.956,..,2.547386592,47.8453125,-2.29568219184875,-1.6939252614975,7.926,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,Uzbekistan,UZB,2017,-1.18751633167267,2.38056145223868,1477884038.18871,-1790139193.7102,2.89668356812091,1798299474.68216,4.39527463456371,...,48.69999885,19.0618147140611,56.811,..,75.9584973,5113.87894595717,-0.244979038834572,-1.13491332530975,5.83,5.83
1448,Uzbekistan,UZB,2018,-1.08239960670471,-6.79621442955791,-3593165923.58668,-622886824.389982,1.1815495684952,624686535.504396,5.87974286685132,...,55.20000067,26.9223953823195,56.697,..,71.57614633,8069.60623655914,-0.304469525814056,-1.09812998771667,5.379,9.35
1449,Uzbekistan,UZB,2019,-1.04689145088196,-5.59228351814186,-3371224441.875,-2313130816.86861,3.84264818715085,2316482962.28621,5.98156194197317,...,70.39999865,17.8148511042514,56.55,..,101.2222235,8836.7875,-0.296665459871292,-1.08138024806976,4.92,8.98
1450,Uzbekistan,UZB,2020,-1.07281696796417,-5.0276459111596,-3027884732.25607,-1716938024.48459,2.86966608446119,1728247827.582,1.995585081437,...,71.09999824,11.4424248275654,55.62,72.687,99.58344787,10054.2613341653,-0.436710476875305,-1.11273300647736,5.29,5.293


In [37]:
# Changing '..' with nan
panel_pivot.replace('..', np.nan, inplace=True)

data = panel_pivot.copy()
cols = list(data.columns)
cols.remove('Country Code')
cols.remove('Country Name')

data['Country_Code'] = data['Country Code']

for column in cols:
    # Удаляем лишние символы, если они есть, заменяем запятые в числах
    data[column] = data[column].replace(',', '', regex=True)
    # Преобразуем в числовой формат
    data[column] = pd.to_numeric(data[column], errors='coerce')

for i in data.columns:
    print(i, data[i].isna().sum())

Country Name 0
Country Code 0
Year 0
Control of Corruption: Estimate 186
Current account balance (% of GDP) 261
Current account balance (BoP, current US$) 257
Foreign direct investment, net (BoP, current US$) 272
Foreign direct investment, net inflows (% of GDP) 207
Foreign direct investment, net inflows (BoP, current US$) 201
GDP growth (annual %) 73
GDP per capita (current US$) 41
Government Effectiveness: Estimate 195
Individuals using the Internet (% of population) 121
Inflation, GDP deflator (annual %) 74
Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate) 198
Labor force with advanced education (% of total working-age population with advanced education) 553
Mobile cellular subscriptions (per 100 people) 70
Official exchange rate (LCU per US$, period average) 463
Political Stability and Absence of Violence/Terrorism: Estimate 174
Rule of Law: Estimate 160
Unemployment, total (% of total labor force) (modeled ILO estimate) 198
Unemployment,

In [38]:
# Taking choosed variables from whole dataset
selected_variables = [
    'Year',
    'Country Name',
    'Country Code',
    # 'Foreign direct investment, net (BoP, current US$)',
    'Foreign direct investment, net inflows (% of GDP)',
    'Control of Corruption: Estimate',
    'Current account balance (% of GDP)',
    'GDP growth (annual %)',
    'GDP per capita (current US$)',
    'Government Effectiveness: Estimate',
    'Individuals using the Internet (% of population)',
    'Inflation, GDP deflator (annual %)',
    'Official exchange rate (LCU per US$, period average)',
    'Political Stability and Absence of Violence/Terrorism: Estimate',
    'Rule of Law: Estimate',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

data = data[selected_variables]

# Missing values

In [39]:
for i in data.columns:
    missing_count = data[i].isna().sum()
    missing_percentage = (missing_count / len(data[i])) * 100
    print(f"{i}: {missing_count} missed values ({missing_percentage:.2f}%)")

Year: 0 missed values (0.00%)
Country Name: 0 missed values (0.00%)
Country Code: 0 missed values (0.00%)
Foreign direct investment, net inflows (% of GDP): 207 missed values (14.26%)
Control of Corruption: Estimate: 186 missed values (12.81%)
Current account balance (% of GDP): 261 missed values (17.98%)
GDP growth (annual %): 73 missed values (5.03%)
GDP per capita (current US$): 41 missed values (2.82%)
Government Effectiveness: Estimate: 195 missed values (13.43%)
Individuals using the Internet (% of population): 121 missed values (8.33%)
Inflation, GDP deflator (annual %): 74 missed values (5.10%)
Official exchange rate (LCU per US$, period average): 463 missed values (31.89%)
Political Stability and Absence of Violence/Terrorism: Estimate: 174 missed values (11.98%)
Rule of Law: Estimate: 160 missed values (11.02%)
Unemployment, total (% of total labor force) (modeled ILO estimate): 198 missed values (13.64%)


In [40]:
# Подсчет пропущенных значений по годам
missing_values_by_country = data.isna().groupby(data['Year']).sum()

# Вывод результата
print(missing_values_by_country)

      Year  Country Name  Country Code  \
Year                                     
2000     0             0             0   
2001     0             0             0   
2002     0             0             0   
2003     0             0             0   
2004     0             0             0   
2005     0             0             0   
2006     0             0             0   
2007     0             0             0   
2008     0             0             0   
2009     0             0             0   
2010     0             0             0   
2011     0             0             0   
2012     0             0             0   
2013     0             0             0   
2014     0             0             0   
2015     0             0             0   
2016     0             0             0   
2017     0             0             0   
2018     0             0             0   
2019     0             0             0   
2020     0             0             0   
2021     0             0          

In [41]:
# Подсчет пропущенных значений по странам
missing_values_by_country = data.isna().groupby(data['Country Name']).sum()

# Вывод результата
print(missing_values_by_country)

                Year  Country Name  Country Code  \
Country Name                                       
Afghanistan        0             0             0   
Albania            0             0             0   
Andorra            0             0             0   
Armenia            0             0             0   
Austria            0             0             0   
...              ...           ...           ...   
Turkiye            0             0             0   
Turkmenistan       0             0             0   
Ukraine            0             0             0   
United Kingdom     0             0             0   
Uzbekistan         0             0             0   

                Foreign direct investment, net inflows (% of GDP)  \
Country Name                                                        
Afghanistan                                                     2   
Albania                                                         0   
Andorra                                        

In [42]:
# Подсчет пропущенных значений по странам
# Сначала получаем DataFrame с True/False для пропущенных значений
missing_values = data.isna()

# Считаем количество True (пропущенных значений) для каждой страны
missing_values_by_country = missing_values.groupby(data['Country Name']).sum()

# Теперь суммируем пропущенные значения по всем колонкам для каждой страны
total_missing_values_by_country = missing_values_by_country.sum(axis=1)

# Вывод результата
print(total_missing_values_by_country.sort_values(by="Name"))


Country Name
Afghanistan       26
Albania            4
Andorra           90
Armenia            4
Austria           31
                  ..
Turkiye            4
Turkmenistan      50
Ukraine            4
United Kingdom     4
Uzbekistan        21
Length: 66, dtype: int64


In [43]:
# Группировка по странам и заполнение методом ffill и bfill
for column in data.columns:
    if data[column].dtype in ['float64', 'int64']:
        # Прямое заполнение
        forward_filled = data.groupby('Country Name')[column].transform(lambda x: x.fillna(method='ffill'))
        # Обратное заполнение
        backward_filled = data.groupby('Country Name')[column].transform(lambda x: x.fillna(method='bfill'))
        
        # Объединяем оба метода, выбирая ближайшее заполненное значение
        data[column] = forward_filled.combine_first(backward_filled)

# Проверка результатов
data.head(2)

/var/folders/97/10g5mgd92035d_qj3kybh7km0000gn/T/ipykernel_98231/4128357613.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forward_filled = data.groupby('Country Name')[column].transform(lambda x: x.fillna(method='ffill'))
/var/folders/97/10g5mgd92035d_qj3kybh7km0000gn/T/ipykernel_98231/4128357613.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  backward_filled = data.groupby('Country Name')[column].transform(lambda x: x.fillna(method='bfill'))
/var/folders/97/10g5mgd92035d_qj3kybh7km0000gn/T/ipykernel_98231/4128357613.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forward_filled = data.groupby('Country Name')[column].transform(lambda x: x.fillna(method='ffill'))
/var/folders/97/10g5mgd92035d_qj3kybh7km0000gn/T

,Year,Country Name,Country Code,"Foreign direct investment, net inflows (% of GDP)",Control of Corruption: Estimate,Current account balance (% of GDP),GDP growth (annual %),GDP per capita (current US$),Government Effectiveness: Estimate,Individuals using the Internet (% of population),"Inflation, GDP deflator (annual %)","Official exchange rate (LCU per US$, period average)",Political Stability and Absence of Violence/Terrorism: Estimate,Rule of Law: Estimate,"Unemployment, total (% of total labor force) (modeled ILO estimate)"
0,2000,Afghanistan,AFG,1.31,-1.27,-2.36,8.83,182.17,-2.17,0.00,11.66,47.36,-2.44,-1.78,7.94
1,2001,Afghanistan,AFG,1.31,-1.27,-2.36,8.83,182.17,-2.17,0.00,11.66,47.50,-2.44,-1.78,7.94


# Functional forms

In [44]:
import seaborn as sns
import matplotlib.pyplot as plt

variables = [
    'Control of Corruption: Estimate',
    'Current account balance (% of GDP)',
    'GDP growth (annual %)',
    'GDP per capita (current US$)',
    'Government Effectiveness: Estimate',
    'Individuals using the Internet (% of population)',
    'Inflation, GDP deflator (annual %)',
    'Official exchange rate (LCU per US$, period average)',
    'Political Stability and Absence of Violence/Terrorism: Estimate',
    'Rule of Law: Estimate',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

In [45]:
# # Переменная для оси X
# x_variable = 'Foreign direct investment, net inflows (% of GDP)'

# # Установка стиля графиков
# # Создание графиков
# for variable in variables:
#     plt.figure(figsize=(10, 6))
#     sns.scatterplot(data=filtered_data, x=x_variable, y=variable)
#     plt.title(f'Relationship between {x_variable} and {variable}')
#     plt.xlabel(x_variable)
#     plt.ylabel(variable)
#     plt.show()

# Model

In [46]:
data['Country_Code'] = data['Country Code']

formula = (
    'Q("Foreign direct investment, net inflows (% of GDP)") ~ '
    'Q("Control of Corruption: Estimate") + '
    'Q("Current account balance (% of GDP)") + '
    'I(Q("GDP growth (annual %)")**3) + '
    'Q("GDP per capita (current US$)") + '
    'Q("Government Effectiveness: Estimate") + '
    'Q("Individuals using the Internet (% of population)") + '
    'Q("Inflation, GDP deflator (annual %)") + '
    # 'Q("Official exchange rate (LCU per US$, period average)") + '
    'Q("Political Stability and Absence of Violence/Terrorism: Estimate") + '
    'Q("Rule of Law: Estimate") + '
    'Q("Unemployment, total (% of total labor force) (modeled ILO estimate)") + '
    'C(Country_Code) + C(Year)'
)


countries_to_keep = ['DEU', 'GBR', 'IRL', 'NLD', 'POL', 'BEL', 'DNK', 'RUS']
filtered_data = data[data['Country_Code'].isin(countries_to_keep)]

In [47]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Создайте модель OLS с робастными ошибками
model = smf.gls(formula, data=filtered_data).fit(cov_type='HC1')

# Выведите сводку модели
print(model.summary())
pd.options.display.float_format = '{:,.2f}'.format

                                              GLS Regression Results                                              
Dep. Variable:     Q("Foreign direct investment, net inflows (% of GDP)")   R-squared:                       0.543
Model:                                                                GLS   Adj. R-squared:                  0.416
Method:                                                     Least Squares   F-statistic:                     14.10
Date:                                                    Thu, 16 May 2024   Prob (F-statistic):           1.38e-31
Time:                                                            14:56:05   Log-Likelihood:                -647.95
No. Observations:                                                     176   AIC:                             1374.
Df Residuals:                                                         137   BIC:                             1498.
Df Model:                                                              38       

In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [49]:
datus = filtered_data[selected_variables].copy()
datus.drop(['Country Name', 'Country Code'], axis=1)

,Year,"Foreign direct investment, net inflows (% of GDP)",Control of Corruption: Estimate,Current account balance (% of GDP),GDP growth (annual %),GDP per capita (current US$),Government Effectiveness: Estimate,Individuals using the Internet (% of population),"Inflation, GDP deflator (annual %)","Official exchange rate (LCU per US$, period average)",Political Stability and Absence of Violence/Terrorism: Estimate,Rule of Law: Estimate,"Unemployment, total (% of total labor force) (modeled ILO estimate)"
176,2000,37.48,1.45,4.50,3.72,"23,098.89",1.66,29.43,2.01,NaN,1.13,1.31,6.59
177,2001,37.26,1.45,4.50,1.10,"23,015.07",1.66,31.29,1.98,NaN,1.13,1.31,6.18
178,2002,7.01,1.46,4.50,1.71,"25,006.19",1.92,46.33,1.64,NaN,1.26,1.34,6.91
179,2003,10.86,1.38,3.40,1.04,"30,655.21",1.87,49.97,1.85,NaN,0.86,1.33,7.68
180,2004,12.05,1.36,3.10,3.57,"35,429.41",1.85,53.86,1.91,NaN,0.71,1.31,7.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,2017,4.68,1.81,-3.50,2.66,"40,572.12",1.43,90.42,1.86,0.78,0.38,1.64,4.33
1426,2018,-0.87,1.81,-3.92,1.40,"43,203.81",1.36,90.69,1.92,0.75,0.08,1.60,4.00
1427,2019,0.69,1.75,-2.69,1.64,"42,662.54",1.45,92.52,2.12,0.78,0.53,1.57,3.74
1428,2020,5.83,1.66,-2.90,-10.36,"40,217.01",1.33,94.82,5.08,0.78,0.48,1.46,4.47


In [50]:
# Проверка пропущенных значений
print(datus.isnull().sum())

# Замена пропущенных значений (например, средними значениями столбцов)
datus = datus.fillna(data.mean())

# Либо удаление строк с пропущенными значениями
datus = datus.dropna()

Year                                                                    0
Country Name                                                            0
Country Code                                                            0
Foreign direct investment, net inflows (% of GDP)                       0
Control of Corruption: Estimate                                         0
Current account balance (% of GDP)                                      0
GDP growth (annual %)                                                   0
GDP per capita (current US$)                                            0
Government Effectiveness: Estimate                                      0
Individuals using the Internet (% of population)                        0
Inflation, GDP deflator (annual %)                                      0
Official exchange rate (LCU per US$, period average)                   88
Political Stability and Absence of Violence/Terrorism: Estimate         0
Rule of Law: Estimate                 

TypeError: Could not convert ['AfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAfghanistanAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraAndorraArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaArmeniaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAustriaAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanAzerbaijanA

In [ ]:
X = filtered_data[selected_variables]

# Добавление константы
X = add_constant(X)

# Расчет VIF
vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif)